In [ ]:
# 환경 초기화 및 라이브러리 재설치
import os

# 1) PyTorch 설치
!pip install torch torchvision torchaudio

# 2) Triton 설치
!pip install triton

# 3) LLM 학습 라이브러리 (버전 고정)
# Transformers 4.46 + PEFT 0.13 + Bitsandbytes 0.44
!pip install transformers==4.46.0 \
             peft==0.13.2 \
             accelerate==1.1.1 \
             bitsandbytes==0.44.1 \
             trl==0.12.0 \
             datasets==3.1.0 \
             huggingface_hub \
             pandas \
             scipy \
             flash-attn

# 설치 후 Kernel 재시작해야 함
print("설치 완료")

In [3]:
import torch
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from huggingface_hub import login

In [ ]:
login(token=os.getenv("HF_TOKEN"))

MODEL_ID = "kakaocorp/kanana-nano-2.1b-instruct"     # 베이스 모델 ID (혹은 경로)
NEW_MODEL_NAME = "kanana-nano-2.1B-customer-emotional" # 저장할 모델명
DATA_FILE = "hana_rewritten.csv"

In [ ]:
# 모델 및 토크나이저 로드
print(f"데이터 로드 중: {DATA_FILE}")
df = pd.read_csv(DATA_FILE)
dataset = Dataset.from_pandas(df)

print("모델 로드 중...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
tokenizer.padding_side = 'right'
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="sdpa",
    torch_dtype=torch.bfloat16
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
# LoRA 설정
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
)

# 학습 설정
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=32,
    gradient_accumulation_steps=2,
    dataloader_num_workers=8,
    optim="paged_adamw_32bit",
    
    save_steps=100,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="none"
)

In [ ]:
# 데이터 포맷팅
def formatting_prompts_func(example):
    output_texts = []
    
    # 데이터셋의 각 행을 순회하며 처리
    for i in range(len(example['counselor_utterance'])):
        # 1. 감정 태그 가져오기 (없으면 '일반'으로 처리)
        # 데이터셋 컬럼명: 'emotion'
        current_emotion = example['emotion'][i] if example['emotion'][i] else "일반"
        
        # 2. 정답 데이터(Target) 가져오기
        # 'customer_utterance_rewritten' 컬럼을 정답으로 사용합니다.
        # (만약 rewritten이 비어있다면, 원본 'customer_utterance'를 사용하도록 안전장치 추가)
        target_response = example['customer_utterance_rewritten'][i]
        if not target_response or str(target_response) == 'nan':
             target_response = example['customer_utterance'][i]

        # 3. 시스템 프롬프트 동적 생성 (핵심!)
        # 감정이 '일반'일 때와 아닐 때를 구분하여 지시사항을 다르게 줍니다.
        if current_emotion == "일반":
            system_msg = "당신은 하나카드의 고객입니다. 상담원의 질문이나 안내에 대해 실제 고객처럼 자연스럽게 응답하세요."
        else:
            # 감정이 있을 경우, 해당 감정을 연기하도록 명시
            system_msg = f"당신은 하나카드의 고객입니다. 현재 당신의 감정 및 성격 상태는 '{current_emotion}'입니다. 이 페르소나에 맞춰 상담원에게 응답하세요."

        # 4. 메시지 구성
        messages = [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": example['counselor_utterance'][i]},
            {"role": "assistant", "content": target_response}
        ]
        
        # 5. 토크나이저로 텍스트 변환
        text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        output_texts.append(text)
        
    return output_texts

In [ ]:
# 학습기 설정
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    formatting_func=formatting_prompts_func,
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_args,
    packing=False,
)

print("학습 시작...")
trainer.train()

# 어댑터 저장
trainer.model.save_pretrained(NEW_MODEL_NAME)
tokenizer.save_pretrained(NEW_MODEL_NAME)

In [ ]:
print("메모리 정리 및 모델 병합 중...")
# del model, trainer
torch.cuda.empty_cache()

# 베이스 모델 재로드 (FP16)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

# 병합
model_to_merge = PeftModel.from_pretrained(base_model, NEW_MODEL_NAME)
merged_model = model_to_merge.merge_and_unload()
print("병합 완료")

# 업로드
print(f"업로드 시작: {NEW_MODEL_NAME}")
merged_model.push_to_hub(NEW_MODEL_NAME, use_temp_dir=False, use_auth_token=True)
tokenizer.push_to_hub(NEW_MODEL_NAME, use_temp_dir=False, use_auth_token=True)

print("✅ 업로드 완료")